# Access & Subset HyP3 SBAS Stack (InSAR or Burst-InSAR)

*Author: Alex Lewandowski; Alaska Satellite Facility*

This notebook assumes that you have already ordered processing for an InSAR Short Baseline Subset (SBAS) stack using [ASF HyP3](https://hyp3-docs.asf.alaska.edu/), available in a web browser at [ASF Vertex](https://search.asf.alaska.edu/) or programmatically with [hyp3-sdk](https://github.com/ASFHyP3/hyp3-sdk).

---

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div>
    <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">
      <img src="github_issues.png" alt="GitHub logo over the word Issues" style="width: 75px; margin-right: 10px;">
    </a>
  </div>
  <div>
    <b>Find a notebook bug or have a feature request?</b>
    <br/>
    Explore current issues on its GitHub repository, add to the discussion, or start a new bug report or feature request: <a href="https://github.com/ASFOpenSARlab/opensarlab_MintPy_Recipe_Book/issues">opensarlab_MintPy_Recipe_Book Issues</a>
  </div>
</div>

<div class="alert alert-info" style="display: flex; align-items: center; justify-content: space-between; font-family: 'Times New Roman', Times, serif; background-color: #d1ecf1;">
  <div>
    <a href="mailto:uso@asf.alaska.edu">
      <img src="ASF_support_logo.png" alt="ASF logo" style="width: 75px; margin-right: 10px;">
    </a>
  </div>
  <div>
    <b>Have a question related to SAR, ASF data access, or performing SBAS time series analyses with MintPy?</b>
    <br/>
    Contact ASF User Support: <a href="mailto:uso@asf.alaska.edu">uso@asf.alaska.edu</a>
  </div>
</div>

---

## 0. Import Required Software 

In [ ]:
from datetime import datetime
from pathlib import Path
import sys
from tqdm.auto import tqdm

import contextily as ctx
import geopandas as gpd
from hyp3_sdk import Batch, HyP3
from ipyfilechooser import FileChooser
from IPython.display import Markdown, display
import matplotlib.pyplot as plt 
from matplotlib.lines import Line2D
import numpy as np
import opensarlab_lib as osl
from osgeo import gdal, ogr
gdal.UseExceptions()
from shapely.geometry import box
from rasterio.warp import transform_bounds

current = Path("..").resolve()
sys.path.append(str(current))
import util.util as util

%matplotlib widget

---

## 1. Select or create a working directory for the analysis

In [ ]:
age = osl.select_parameter(
    [
        "Access a new SBAS stack",
        "Add to existing SBAS stack"
    ]
)
display(age)

In [ ]:
new = 'new' in age.value

if new:
    print(f'Current working directory: {Path.cwd()}')
    print('Create a new directory to hold your data:')
    data_path = input(f'Enter an unused path for a new data directory:  {Path.home()}/')
    try:
        data_path = Path.home() / data_path.strip()
        data_path.mkdir()
    except:
        raise
else:
    path = Path.home()
    fc = FileChooser(path)
    display(fc)

In [ ]:
if not new:
    data_path = Path.home()/fc.selected_path

---

## 2. Migrate SBAS Stack from HyP3

**Create a HyP3 object and authenticate**

In [ ]:
hyp3 = HyP3(prompt=True)

**You may search for InSAR projects in your own account or migrate data from any user's account**

- Retrieving data from another user's account only requires their username and the project name.
- It does **not** require the other user's password. 

In [ ]:
hyp3_project = osl.select_parameter(
    [
        'Access InSAR data with any valid HyP3 username and HyP3 Project Name',
        'Search your Projects for available InSAR data'
    ]
)
display(hyp3_project)

**Select your SBAS stack's HyP3 product type**

In [ ]:
product_select = osl.select_parameter(
    [
        'INSAR_GAMMA',
        'INSAR_ISCE_BURST'
    ]
)
print("Select your SBAS stack's HyP3 product type")
display(product_select)

In [ ]:
product_type = product_select.value

In [ ]:
search = "Search" in hyp3_project.value
if search:
    my_hyp3_info = hyp3.my_info()
    active_projects = dict()
    
    print("Checking all HyP3 projects for current INSAR_GAMMA jobs")
    for project in tqdm(my_hyp3_info['job_names']):
            batch = Batch()
            batch = hyp3.find_jobs(
                name=project, 
                job_type=product_type
            ).filter_jobs(running=False, include_expired=False)
            if len(batch) > 0:
                active_projects.update({batch.jobs[0].name: batch})
    
    if len(active_projects) > 0:
        display(Markdown("<text style='color:darkred;'>Note: After selecting a project, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
        display(Markdown("<text style='color:darkred;'>Otherwise, you will rerun this code cell.</text>"))
        print('\nSelect a Project:')
        project_select = osl.select_parameter(active_projects.keys())
        display(project_select)
    else:
        print("Found no active projects containing InSAR products")
else:
    username = input("enter the HyP3 username on the account containing an SBAS stack to migrate")
    project_name = input("Enter the HyP3 project name")
    batch = Batch()
    batch = hyp3.find_jobs(
        name=project_name, 
        job_type=product_type, 
        user_id=username
    ).filter_jobs(running=False, include_expired=False)

**Select a date range of products to migrate:**

In [ ]:
if search:
    jobs = active_projects[project_select.value]
else:
    jobs = batch

display(Markdown("<text style='color:darkred;'>Note: After selecting a date range, you should select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you may simply rerun this code cell.</text>"))
print('\nSelect a Date Range:')
dates = osl.get_job_dates(jobs)
date_picker = osl.gui_date_picker(dates)
display(date_picker)

**Save the selected date range and remove products falling outside of it:**

In [ ]:
date_range = osl.get_slider_vals(date_picker)
date_range[0] = date_range[0].date()
date_range[1] = date_range[1].date()
print(f"Date Range: {str(date_range[0])} to {str(date_range[1])}")
jobs = osl.filter_jobs_by_date(jobs, date_range)

**Gather the available paths and orbit directions for the remaining products:**

In [ ]:
display(Markdown("<text style='color:darkred;'><text style='font-size:150%;'>This may take some time for projects containing many jobs...</text></text>"))
osl.set_paths_orbits(jobs)
paths = set()
for p in jobs:
    paths.add(p.path)
display(Markdown(f"<text style=color:blue><text style='font-size:175%;'>Done.</text></text>"))

**Select a path:**

- Sentinel-1 has a 12-day repeat cycle so it is not appropriate to merge interferograms across multiple orbital paths. 
- If multiple paths are represented in the SBAS stack, select one.

In [ ]:
display(Markdown("<text style='color:darkred;'>Note: After selecting a path, you must select the next cell before hitting the 'Run' button or typing Shift/Enter.</text>"))
display(Markdown("<text style='color:darkred;'>Otherwise, you will simply rerun this code cell.</text>"))
print('\nSelect a Path:')
path_choice = osl.select_parameter(paths)
display(path_choice)

**Save the selected flight path/s:**

In [ ]:
flight_path = path_choice.value
if flight_path:
    if flight_path:
        print(f"Flight Path: {flight_path}")
    else:
        print('Flight Path: All Paths')
else:
    print("WARNING: You must select a flight path in the previous cell, then rerun this cell.")

**Filter jobs by path:**

In [ ]:
jobs = osl.filter_jobs_by_path(jobs, [flight_path])
print(f"There are {len(jobs)} products to migrate.")

**Migrate the products, unzip them into a directory named after the product type, and delete the zip files:**

In [ ]:
print(f"\nProject: {jobs.jobs[0].name}")
project_zips = jobs.download_files(data_path)
for z in project_zips:
    osl.asf_unzip(str(data_path), str(z))
    z.unlink()

---

## 3. Confirm Presence of a DEM, Azimuth Angle Map, and Incidence Angle Map

- These are optional addon products for HyP3, which are necessary for MintPy
    - Incidence angle maps are included with HyP3 jobs when the `Include Look Vectors` option is selected.
    - DEMs are included with HyP3 jobs when the `Include DEM` option is selected
- This is an optional addon product for HyP3, which is necessary for MintPy if running the correct_SET (Solid Earth Tides) step
    - Azimuth angle maps are included with HyP3 jobs when the `Include Look Vectors` option is selected

**All of the above mentioned files will be included in an InSAR project if Set MintPy Options is selected when adding InSAR jobs to a project in ASF-Search (Vertex)**

In [ ]:
dem = sorted(list(data_path.glob('*/*dem*.tif')))
lv_phi = sorted(list(data_path.glob('*/*lv_phi*.tif')))
lv_theta = sorted(list(data_path.glob('*/*lv_theta*.tif')))
water_mask = sorted(list(data_path.glob('*/*_water_mask*.tif')))
unw = sorted(list(data_path.glob('*/*_unw_phase*.tif')))
corr = sorted(list(data_path.glob('*/*_corr*.tif')))
conn_comp = sorted(list(data_path.glob('*/*_conncomp*.tif')))
tiff_path = dem + lv_phi + lv_theta + water_mask + unw + corr + conn_comp

if len(dem) > 0:
    print("Success: Found at least 1 DEM.")
else:
    raise FileNotFoundError("Failed to find at least 1 DEM.\n"
                            "You will not be able to successfully run a MintPy time-series unless you"
                            "reorder your HyP3 project with DEMS or provide one from another source.")
                            
if len(lv_phi) > 0:
    print("Success: Found at least 1 lv_phi look vector file.")
else:
    raise FileNotFoundError("Failed to find at least 1 lv_phi look vector file.\n"
                            "You will not be able to successfully run a MintPy time-series unless your"
                            "reorder your HyP3 project with 'Include Look Vectors' option selected.")
    
if len(lv_theta) > 0:
    print("Success: Found at least 1 lv_theta look vector file.")
else:
    raise FileNotFoundError("Failed to find at least 1 lv_theta look vector file.\n"
                            "You will not be able to successfully run a MintPy time-series unless your"
                            "reorder your HyP3 project with 'Include Look Vectors' option selected.")

In [ ]:
gdf = gpd.GeoDataFrame(
    {
    'tiff_path': tiff_path,
    'EPSG': [util.get_epsg(p) for p in tiff_path],
    'geometry': [util.get_geotiff_bbox(p) for p in tiff_path],
    }
                  )
display(gdf)
if gdf['EPSG'].nunique() > 1:
    raise Exception(f'Error: Data is in {gdf["EPSG"].nunique()} projections.')

---
## 4. Subset the Stack

**Select how to define your AOI for subsetting**

In [ ]:
subset_option = osl.select_parameter([
    'Draw a bounding box on a map',
    'Provide a polygon in Well-Known Text (WKT)',
    'Provide a shapefile'
])

display(subset_option)

**Determine the maximum and common extents of the stack and plot an Area-of_Interest Selector:**

In [ ]:
draw = 'Draw' in subset_option.value
wkt_poly = 'WKT' in subset_option.value

max_extents = osl.get_max_extents(unw)
xmin, ymin, xmax, ymax = transform_bounds(int(osl.get_projection(str(unw[0]))), 3857, *max_extents)
max_extents = [xmin, ymin, xmax, ymax]

common_extents = osl.get_common_coverage_extents(unw)
xmin, ymin, xmax, ymax = transform_bounds(int(osl.get_projection(str(unw[0]))), 3857, *common_extents)
common_extents = [xmin, ymin, xmax, ymax]

if draw:   
    print('Maximum Extents: Pixels in this area are guaranteed to be included in at least one RTC in the stack.\n\n')
    
    print('Common Extents: Pixels in this area are guaranteed to included in every RTC in the stack.\n\n')
    
    print(f"Select an AOI inside the common area covered by the stack.")
    
    aoi = osl.AOI_Selector(max_extents, common_extents, figsize=(10, 8))
elif wkt_poly:   
    correct_wkt_input = False
    while not correct_wkt_input:
        wkt, wkt_shapely_geom = util.get_valid_wkt()
        wkt_ogr_geom = ogr.CreateGeometryFromWkt(wkt)
        
        if not util.check_within_bounds(wkt_shapely_geom, gdf):
            print('WKT exceeds bounds of at least one dataset')
            continue
    
        correct_wkt_input = True
        
    shp_path = data_path / f'shape_{datetime.strftime(datetime.now(), "%Y%m%dT%H%M%S")}.shp'
    epsg = int(gdf.iloc[0]['EPSG'])
    util.save_shapefile(wkt_ogr_geom, epsg, shp_path)
else:
    print('Select a shapefile (*.shp)')
    shp_fc = FileChooser(Path.home())
    display(shp_fc)

**If providing an AOI with WKT or a shapefile, confirm its location on a plot before subsetting**

In [ ]:
if not draw and not wkt_poly:
    shp_path = Path(shp_fc.selected)
    if shp_path.suffix != '.shp':
        raise Exception(f'Selected file suffix not ".shp"')
if not draw:   
    shp_gdf = gpd.read_file(shp_path)

    box1 = gpd.GeoDataFrame({"geometry": [box(*max_extents)]}, crs='EPSG:3857')
    box2 = gpd.GeoDataFrame({"geometry": [box(*common_extents)]}, crs='EPSG:3857')
    box1 = box1.to_crs(crs=shp_gdf.crs.to_string())
    box2 = box2.to_crs(crs=shp_gdf.crs.to_string())
    
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    shp_gdf.plot(ax=ax, color='blue', alpha=0.5, edgecolor='k')
    box1.plot(ax=ax, color='none', edgecolor='red', linewidth=4)
    box2.plot(ax=ax, color='none', edgecolor='green', linewidth=4)
    
    ctx.add_basemap(ax, crs=shp_gdf.crs.to_string(), source=ctx.providers.OpenTopoMap)
    
    minx, miny, maxx, maxy = box1.total_bounds
    buffer_percentage = 0.05
    buffer_x = (maxx - minx) * buffer_percentage
    buffer_y = (maxy - miny) * buffer_percentage
    buffered_minx = minx - buffer_x
    buffered_maxx = maxx + buffer_x
    buffered_miny = miny - buffer_y
    buffered_maxy = maxy + buffer_y
    
    ax.set_xlim(buffered_minx, buffered_maxx)
    ax.set_ylim(buffered_miny, buffered_maxy)
    ax.set_title('Loaded Shape for Subsetting')
    
    legend_elements = [
        Line2D([0], [0], color='red', lw=4, label='Max area covered by data stack'),
        Line2D([0], [0], color='green', lw=4, label='Common area covered by data stack')
    ]
    ax.legend(handles=legend_elements)
    plt.show()
    print('Max area covered by data stack: Every pixel guaranteed to be present in at least 1 image')
    print('Common area covered by data stack: Every pixel guaranteed to be present in all images')

**Subset the data** 

In [ ]:
for pth in tqdm(gdf['tiff_path']):
    print(f'Subsetting: {pth}')

    if draw:
        try:
            xmin, ymin, xmax, ymax = transform_bounds(
                3857,
                int(gdf['EPSG'].iloc[0]), 
                *[aoi.x1, aoi.y1, aoi.x2, aoi.y2]
            )
            ul = [xmin, ymax]
            lr = [xmax, ymin]
        except TypeError:
            print('TypeError')
            display(Markdown(f'<text style=color:red>This error may occur if an AOI was not selected.</text>'))
            display(Markdown(f'<text style=color:red>Note that the square tool icon in the AOI selector menu is <b>NOT</b> the selection tool. It is the zoom tool.</text>'))
        temp_pth = pth.parent/f'subset_{pth.name}'
        gdal.Translate(destName=str(temp_pth), srcDS=str(pth), projWin=[ul[0], ul[1], lr[0], lr[1]])
        pth.unlink()
        temp_pth.rename(pth)
    else:
        warp_args = gdal.WarpOptions(cutlineDSName=shp_path, cropToCutline=True) 
        gdal.Warp(str(pth), str(pth), options=warp_args)

**Select whether to convert to WGS84 (lat/lon) prior to loading into MintPy**

- HyP3 data are delivered in UTM.
- This notebook intentionally only converts to WGS84, not back to UTM.
- Repeatedly converting data back and forth between coordinate systems will lead to a loss in precision.
- If you have already converted the data to WGS84 and wish to go back to UTM, delete the data and rerun this notebook to access fresh copies from HyP3.

In [ ]:
coord_sys_option = osl.select_parameter(['UTM', 'WGS84 (lat/lon)'], 'Select a coordinate system for your data')
display(coord_sys_option)

**If desired, convert to WGS84**

In [ ]:
wgs84 = 'WGS84' in coord_sys_option.value
if wgs84:
    for pth in tqdm(gdf['tiff_path']):
        print(f'Converting {pth} to WGS84')
        gdal.Warp(str(pth), str(pth), dstSRS='EPSG:4326')

**Remove any subset scenes containing no data:**

In [ ]:
removed = []
for pth in tqdm(gdf['tiff_path']):
    raster = gdal.Open(str(pth))
    if raster:
        band = raster.ReadAsArray()
        if np.count_nonzero(band) < 1:
            pth.unlink()
            removed.append(pth)

if len(removed) == 0:
    print("No Geotiffs were removed")
else:
    print(f"{len(removed)} GeoTiffs removed:")
    for pth in removed:
        print(pth)